In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import spacy
import re
import os
import nltk



#### Create placeholders for results

In [ ]:
clin_concepts_set = []
clin_classes_set_prelim  = []

### Aggregate mentions of clinical concept (i.e. 1 number per consultation)
clin_concepts_number_per_consultation = {}

### Aggregate mentions of each clinical classification (separately) (i.e. >1 number per consultation)
clin_classes_type_number_per_consultation = {}

#### Create a dictionary linking each clinical concept to its classification

In [ ]:
%run -i  create_clin_concepts_dict.py

print("Number of concepts removed: {}".format(num_concepts_to_remove))

print("Original length of clin concepts dictionary: {}".format(original_len_clin_concepts_dict))
print("Revised length of clin concepts dictionary: {}".format(len(clin_concepts_dict)))

clin_concepts_dict

### Prepare the tokeniser

In [ ]:
%run -i sentiment_tokenize.py


### Create list of all classes (for dictionary of count of classes)

In [ ]:
list_classes = ['BIOSUB',
    'BODYLOC',
    'BODYMEAS',
    'DDF',
    'DIAGPROC',
    'DISORDER',
    'DOSEUNIT',
    'DRT',
    'DRUG',
    'FREQ',
    'LABTEST',
    'NUM',
    'PROBLEM',
    'THERPROCDEV',
    'TK',
    'TOD',
    'TUNIT',
    'UNIT',
    'RUT']
    

### START LOOP HERE - OVER EACH TRANSCRIPT SEPARATELY

In [ ]:
counter = 0
folder  = "sentiment_text_data"

number_of_files = len(os.listdir(folder))
print("Number of files in the directory: {}".format(number_of_files))

while counter < number_of_files:
    
    ### print file name and number:
    file = os.listdir(folder)[counter]
    print()
    print("File number: {}".format(counter + 1))
    print("File name: {}".format(file))
    print()
    
    ### tokenise
    tokens = open_and_tokenize(counter, folder)
    type(tokens)

    ### Dictionary to count no. times each class occurs.(will insert into "clin_classes_type_number_per_consultation" for every consultation)
    count_each_class_type = {}
    for x in list_classes:
        count_each_class_type[x] = 0
    
    # XXX - just so it runs quickly during development.
    # my_text_strings = tokens[0:400]
    my_text_strings = tokens
    
    ### placeholders for results (separate for each consultation)
    classifications_length = 0
    concepts_length        = 0
    all_classifications_consultation_prelim = []

    ### counter for the "for" loop over each token below
    token_number = 1

    ### loop over each token
    for mystring in my_text_strings:
        
        ### store the class & concept (if the token is a "clinical concept")
        classifications_consultation = [value for key, value in clin_concepts_dict.items() if mystring == key.lower()]
        concepts_consultation        = [key for key, value in clin_concepts_dict.items() if mystring == key.lower()]       

        ### add to the master set of concepts / classes across all transcripts.
        clin_classes_set_prelim.extend(classifications_consultation)    
        clin_concepts_set.extend(concepts_consultation)
            # by using "extend", each item added is a STRING. if use "append", an item added at the end 
            # would be a full list (could not use "set" on it)

        ### update the number of clinical concepts per consultation
        classifications_length += len(classifications_consultation)
        concepts_length        += len(concepts_consultation)
        
        ### add the "classification" (e.g. treatment / problem) to the list of classifications for the specific transcript
        if classifications_consultation !=  []:
            all_classifications_consultation_prelim.extend(classifications_consultation)
        
        ### just a simple progress indicator for the token loop
        if token_number % 200 == 0:
            print("Progress tracking: File number {}, Token number {} (of a total of {} tokens)".format(counter+1, token_number, len(tokens)))
        
        ### and back to start of token loop
        token_number += 1
    
    
    print("")
    print("all_classifications_consultation_prelim: before split on hyphen")
    print(all_classifications_consultation_prelim[0:50])

    ### some tokens have multiple classes (e.g. "treatment-problem") - split on the hyphen to account for each class separately.
    all_classifications_consultation_prelim  = [i.split('-', 10)[:] for i in all_classifications_consultation_prelim]

    ### check that the split occurred ok and nothing is missing.
    print("")
    print("all_classifications_consultation_prelim: after split on hyphen")
    all_classifications_consultation_prelim[0:50]

    ### the final container for the classes for a consultation (i.e. put it in the right List form)
    all_classifications_consultation_final = []
    for x in all_classifications_consultation_prelim:
        all_classifications_consultation_final.extend(x)
    ### just show the first 20 (this confirms the content and the appropriate object type)
    all_classifications_consultation_final[0:20]

    ### put info from list into Dictionary format (for this particular consultation)
    for x in all_classifications_consultation_final:
        count_each_class_type[x] +=1 
    print()
    print("count_each_class_type: i.e. the dictionary containing the number of each class for a consultation")
    count_each_class_type
        
        
    print()
    print("classifications_length: for one transcript")
    classifications_length
    print()
    print("concepts_length: for one transcript")
    concepts_length
    
        ## number of times any clinical concept mentioned: add to master dictionary for all transcripts
    clin_concepts_number_per_consultation[str(counter+1)] = concepts_length
        ## number of times each type of clinical classification mentioned: add to master dictionary for all transcripts
    clin_classes_type_number_per_consultation[str(counter+1)] = count_each_class_type
    
    counter  +=  1


### After loop -  look at results.

In [ ]:
type(clin_classes_set_prelim)

### get rid of duplicates
clin_classes_set_prelim = set(clin_classes_set_prelim)

### put in format for splitting on hyphens (i.e. some tokens have multiple classes attached - account for them separately)
clin_classes_set_prelim = list(clin_classes_set_prelim)
print("")
print("Preliminary")
clin_classes_set_prelim

clin_classes_set_prelim = [i.split('-', 10)[:] for i in clin_classes_set_prelim]

clin_classes_set_final  = []
for x in clin_classes_set_prelim:
    clin_classes_set_final.extend(x)

print("")
print("Final")
## Set
clin_classes_set_final  =  set(clin_classes_set_final)
clin_classes_set_final

In [ ]:
## Set
clin_concepts_set =  set(clin_concepts_set)

print("clin_classes_set")
clin_classes_set_final

print("Number of clinical concepts per consultation")
clin_concepts_number_per_consultation

print("How many times each classification appeared in each transcript")
clin_classes_type_number_per_consultation

print("clin_concepts_set")
clin_concepts_set
print()